In [ ]:
# we need the following libraries, so let's install them
!pip install boto3
!pip install torchxrayvision
!pip install SimpleITK
!pip install radiomics
!pip install pyradiomics
!pip install torchvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# importing libraries
from time import time
import multiprocessing
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
from glob import glob
import skimage
import torch
import SimpleITK as sitk
import skimage, torch, torchvision
import torchxrayvision as xrv
import torchvision
import torchxrayvision as xrv
import matplotlib.pyplot as plt
# importing written classes and methods
import sys
from radiomics import featureextractor
from radiomics import firstorder, glcm, glrlm, glszm, shape, shape2D, ngtdm, gldm
import warnings
import logging

warnings.filterwarnings("ignore", category=DeprecationWarning)

# set level for all classes
logger = logging.getLogger("radiomics")
logger.setLevel(logging.ERROR)
# # ... or set level for specific class
# logger = logging.getLogger("radiomics.glcm")
# logger.setLevel(logging.ERROR)


In [ ]:
# here we call the directories that images are saved; features are saved; targets are saved.
directory_ = '/content/sample_data/NIH-ChestXray-14/'
directory_processed_images = directory_ + 'processed_images'
directory_filterss = directory_ + 'filters'
directory_segments = directory_ + 'segments'
directory_cleaned_targets = directory_ + 'cleaned_targets'
directory_features = directory_ + 'features'

In [ ]:
# loading the file of target
target = np.load(directory_cleaned_targets + '/' + 'new_targets.npy',allow_pickle=True).item()

#visuaizing the target
target_data_frame = pd.DataFrame(target)
name_of_targets = list(target_data_frame)
target_data_frame.head(10)

,index,Image Index,Patient ID,Patient Age,Patient Gender,View Position,No Finding,Atelectasis,Cardiomegaly,Consolidation,...,Effusion,Emphysema,Fibrosis,Hernia,Infiltration,Mass,Nodule,Pleural_Thickening,Pneumonia,Pneumothorax
0,0,00000001_000.png,1,58,M,PA,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1,00000001_001.png,1,58,M,PA,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
2,2,00000001_002.png,1,58,M,PA,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
3,3,00000002_000.png,2,81,M,PA,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,00000003_000.png,3,81,F,PA,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
5,5,00000003_001.png,3,74,F,PA,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
6,6,00000003_002.png,3,75,F,PA,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
7,7,00000003_003.png,3,76,F,PA,0,0,0,0,...,0,0,0,1,1,0,0,0,0,0
8,8,00000003_004.png,3,77,F,PA,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
9,9,00000003_005.png,3,78,F,PA,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [ ]:
# total number of targets:
n_samples_target, n_targets_types = target_data_frame.shape
n_samples_target, n_targets_types

(112120, 21)

In [ ]:
# checkin the frontal or lateral type of images:
pd.unique(target_data_frame['View Position'])

array(['PA', 'AP'], dtype=object)

In [ ]:
# total number of unique subjects:
n_subj = len(pd.unique(target_data_frame['Patient ID']))
print("The number of subjects are: {0}".format(n_subj))

The number of subjects are: 30805


In [ ]:
# getting the name of all feature radiomics:
radiomics_names = glob(directory_features + '/*.npy')
print("The number of samples (images) are: {0}".format(len(radiomics_names)))

The number of samples (images) are: 112120


In [ ]:
# getting the number of feature radiomics per image:
n_type_of_features = np.load('/content/sample_data/NIH-ChestXray-14/features/00028932_001__features.npy').shape[0]
n_type_of_features

464

In [ ]:
# preallocating X,Y matrics
Features_names = ['feature_' + str(i) for i in range(n_type_of_features)]
Synced_data = pd.DataFrame( np.ones((n_samples_target, n_type_of_features + n_targets_types))*np.nan,columns = Features_names + name_of_targets)

In [ ]:
# syncing the data
for j in tqdm(range(len(radiomics_names))):
  j_th_featres_name = radiomics_names[j]
  feature_jth = np.load(j_th_featres_name)
  image_index = j_th_featres_name.split('/')[-1].split('__')[0]
  Synced_data.iloc[j, :464] = feature_jth
  Synced_data.iloc[j, 464:] = target_data_frame[target_data_frame['Image Index'] == image_index + '.png']


100%|██████████| 112120/112120 [8:53:10<00:00,  3.50it/s]


In [ ]:
synced_data_loc = directory_ + 'synced_data'
try:
  os.mkdir(synced_data_loc)
except:
  pass
Synced_data.to_csv(synced_data_loc + '.csv')